Having a look at the data

In [1]:
# import packages

import numpy as np
import pandas as pd
pd.options.display.float_format = '{:20,.2f}'.format # disabling scientific notation in pandas
import pickle
import matplotlib.pyplot as plt
import seaborn as sns


/home/slawa/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
train_df_cat = pd.read_csv("data/train_categorial_features_moda.csv")

/tmp/ipykernel_9544/745967132.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df_cat = pd.read_csv("data/train_categorial_features_moda.csv")


In [3]:
train_df_cat.shape

(714167, 14)

In [4]:
train_df_cat.head()

,fullVisitorId,device_browser,device_operatingSystem,geoNetwork_city,geoNetwork_metro,geoNetwork_country,geoNetwork_networkDomain,geoNetwork_region,trafficSource_adContent,trafficSource_campaign,trafficSource_keyword,trafficSource_source,trafficSource_referralPath,weekday
0,10278554503158,0.00,1.00,1.00,0.00,68.00,397.00,1.00,0.00,0.00,0.00,0.00,0.00,3.00
1,20424342248747,0.00,0.00,45.00,0.00,42.00,"6,726.00",37.00,0.00,0.00,0.00,0.00,0.00,2.00
2,27376579751715,0.00,1.00,1.00,0.00,33.00,90.00,1.00,0.00,0.00,0.00,0.00,0.00,5.00
3,39460501403861,0.00,0.00,1.00,0.00,11.00,133.00,1.00,0.00,0.00,0.00,25.00,72.00,0.00
4,40862739425590,0.00,1.00,110.00,2.00,33.00,109.00,21.00,0.00,1.00,13.00,0.00,0.00,5.00


In [5]:
train_df_cat.dtypes

fullVisitorId                  object
device_browser                float64
device_operatingSystem        float64
geoNetwork_city               float64
geoNetwork_metro              float64
geoNetwork_country            float64
geoNetwork_networkDomain      float64
geoNetwork_region             float64
trafficSource_adContent       float64
trafficSource_campaign        float64
trafficSource_keyword         float64
trafficSource_source          float64
trafficSource_referralPath    float64
weekday                       float64
dtype: object

In [6]:
train_df_filter = pd.read_csv("data/train_filtered.csv")

/tmp/ipykernel_9544/2683091660.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df_filter = pd.read_csv("data/train_filtered.csv")


In [7]:
train_df_filter.shape

(902755, 32)

In [8]:
train_df_filter.head()

,channelGrouping,date,fullVisitorId,visitNumber,visitStartTime,totals_bounces,totals_hits,totals_newVisits,totals_pageviews,totals_transactionRevenue,...,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_campaign,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source
0,Organic Search,20160902,1131660440785968503,1,1472830385,1.00,1,1.00,1.00,0.00,...,@,1,0.00,@,@,0,@,organic,@,google
1,Organic Search,20160902,377306020877927890,1,1472880147,1.00,1,1.00,1.00,0.00,...,@,1,0.00,@,@,0,@,organic,@,google
2,Organic Search,20160902,3895546263509774583,1,1472865386,1.00,1,1.00,1.00,0.00,...,@,1,0.00,@,@,0,@,organic,@,google
3,Organic Search,20160902,4763447161404445595,1,1472881213,1.00,1,1.00,1.00,0.00,...,@,1,0.00,@,@,0,google + online,organic,@,google
4,Organic Search,20160902,27294437909732085,2,1472822600,1.00,1,0.00,1.00,0.00,...,@,1,0.00,@,@,1,@,organic,@,google


In [9]:
train_df_filter.dtypes

channelGrouping                                  object
date                                              int64
fullVisitorId                                    object
visitNumber                                       int64
visitStartTime                                    int64
totals_bounces                                  float64
totals_hits                                       int64
totals_newVisits                                float64
totals_pageviews                                float64
totals_transactionRevenue                       float64
device_browser                                   object
device_deviceCategory                            object
device_isMobile                                   int64
device_operatingSystem                           object
geoNetwork_city                                  object
geoNetwork_continent                             object
geoNetwork_country                               object
geoNetwork_metro                                

In [10]:
train_df_flat = pd.read_csv("data/train_flat.csv")

/tmp/ipykernel_9544/1220889983.py:1: DtypeWarning: Columns (2,49) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df_flat = pd.read_csv("data/train_flat.csv")


In [11]:
train_df_flat.shape

(903653, 55)

In [12]:
train_df_flat.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,totals_bounces,totals_hits,...,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_campaign,trafficSource_campaignCode,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,0,1472830385,1,1472830385,1.00,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organic,NaN,google
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,0,1472880147,1,1472880147,1.00,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organic,NaN,google
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,0,1472865386,1,1472865386,1.00,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,organic,NaN,google
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,0,1472881213,1,1472881213,1.00,1,...,NaN,NaN,NaN,NaN,NaN,NaN,google + online,organic,NaN,google
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,0,1472822600,2,1472822600,1.00,1,...,NaN,NaN,NaN,NaN,NaN,True,NaN,organic,NaN,google


In [13]:
train_df_flat.dtypes

channelGrouping                                       object
date                                                   int64
fullVisitorId                                         object
sessionId                                             object
socialEngagementType                                   int64
visitId                                                int64
visitNumber                                            int64
visitStartTime                                         int64
totals_bounces                                       float64
totals_hits                                            int64
totals_newVisits                                     float64
totals_pageviews                                     float64
totals_transactionRevenue                            float64
totals_visits                                          int64
device_browser                                        object
device_browserSize                                   float64
device_browserVersion   

In [14]:
train_df_cat.isna().sum()

fullVisitorId                 0
device_browser                0
device_operatingSystem        0
geoNetwork_city               0
geoNetwork_metro              0
geoNetwork_country            0
geoNetwork_networkDomain      0
geoNetwork_region             0
trafficSource_adContent       0
trafficSource_campaign        0
trafficSource_keyword         0
trafficSource_source          0
trafficSource_referralPath    0
weekday                       0
dtype: int64

In [15]:
train_df_filter.isna().sum() # train_df_filter seems to be the processed data after removing redundant and allegedly less meaningful features

channelGrouping                                 0
date                                            0
fullVisitorId                                   0
visitNumber                                     0
visitStartTime                                  0
totals_bounces                                  0
totals_hits                                     0
totals_newVisits                                0
totals_pageviews                                0
totals_transactionRevenue                       0
device_browser                                  0
device_deviceCategory                           0
device_isMobile                                 0
device_operatingSystem                          0
geoNetwork_city                                 0
geoNetwork_continent                            0
geoNetwork_country                              0
geoNetwork_metro                                0
geoNetwork_networkDomain                        0
geoNetwork_region                               0


In [16]:
train_df_flat.isna().sum()

channelGrouping                                           0
date                                                      0
fullVisitorId                                             0
sessionId                                                 0
socialEngagementType                                      0
visitId                                                   0
visitNumber                                               0
visitStartTime                                            0
totals_bounces                                       453023
totals_hits                                               0
totals_newVisits                                     200593
totals_pageviews                                        100
totals_transactionRevenue                            892138
totals_visits                                             0
device_browser                                            8
device_browserSize                                   903653
device_browserVersion                   

In [17]:
train_df_flat.isna().sum() >= 453023

channelGrouping                                      False
date                                                 False
fullVisitorId                                        False
sessionId                                            False
socialEngagementType                                 False
visitId                                              False
visitNumber                                          False
visitStartTime                                       False
totals_bounces                                        True
totals_hits                                          False
totals_newVisits                                     False
totals_pageviews                                     False
totals_transactionRevenue                             True
totals_visits                                        False
device_browser                                       False
device_browserSize                                    True
device_browserVersion                                 Tr

Note on the values of totals_transactionRevenue: Total transaction revenue is expressed as the value passed to Analytics multiplied by 10^6 (e.g., 2.40 would be given as 2400000).

How many unique visitors did the ecommerce-shop have for the observed period?

In [18]:
unique_customers = train_df_filter.fullVisitorId.nunique()

In [112]:
unique_customers

723854

How many of those generated revenue?

In [19]:
train_df_filter[['fullVisitorId', 'totals_transactionRevenue']].sort_values('totals_transactionRevenue',ascending=False)

,fullVisitorId,totals_transactionRevenue
374904,1957458976293878100,"23,129,500,000.00"
430432,1957458976293878100,"17,855,500,000.00"
380637,5632276788326171571,"16,023,750,000.00"
772974,9417857471295131045,"10,589,140,000.00"
890691,1957458976293878100,"8,677,830,000.00"
...,...,...
302842,6705206951405647737,0.00
302843,262472272057446420,0.00
302844,3627326284569000152,0.00
302845,996291448451917297,0.00


In [20]:
rev_by_customer = train_df_filter[['fullVisitorId', 'totals_transactionRevenue']].groupby('fullVisitorId').sum().sort_values('totals_transactionRevenue',ascending=False)

In [21]:
rev_by_customer_real = rev_by_customer

In [22]:
rev_by_customer

,totals_transactionRevenue
fullVisitorId,
1957458976293878100,"75,916,690,000.00"
5632276788326171571,"16,023,750,000.00"
9417857471295131045,"15,170,120,000.00"
4471415710206918415,"11,211,100,000.00"
4984366501121503466,"9,513,900,000.00"
...,...
2793635141970553946,0.00
2793638325076213828,0.00
2793639349609627123,0.00


In [23]:
rev_by_customer_real.totals_transactionRevenue = rev_by_customer_real.totals_transactionRevenue/(10**6)

In [24]:
rev_by_customer_real

,totals_transactionRevenue
fullVisitorId,
1957458976293878100,"75,916.69"
5632276788326171571,"16,023.75"
9417857471295131045,"15,170.12"
4471415710206918415,"11,211.10"
4984366501121503466,"9,513.90"
...,...
2793635141970553946,0.00
2793638325076213828,0.00
2793639349609627123,0.00


In [25]:
paying_customers = (rev_by_customer.totals_transactionRevenue > 0).sum()

In [26]:
conversion_rate = paying_customers/unique_customers

Only 1.4% of all visitors generate (tracked) revenue (assuming missing values mean no revenue)

In [27]:
train_df_filter.head()

,channelGrouping,date,fullVisitorId,visitNumber,visitStartTime,totals_bounces,totals_hits,totals_newVisits,totals_pageviews,totals_transactionRevenue,...,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_campaign,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source
0,Organic Search,20160902,1131660440785968503,1,1472830385,1.00,1,1.00,1.00,0.00,...,@,1,0.00,@,@,0,@,organic,@,google
1,Organic Search,20160902,377306020877927890,1,1472880147,1.00,1,1.00,1.00,0.00,...,@,1,0.00,@,@,0,@,organic,@,google
2,Organic Search,20160902,3895546263509774583,1,1472865386,1.00,1,1.00,1.00,0.00,...,@,1,0.00,@,@,0,@,organic,@,google
3,Organic Search,20160902,4763447161404445595,1,1472881213,1.00,1,1.00,1.00,0.00,...,@,1,0.00,@,@,0,google + online,organic,@,google
4,Organic Search,20160902,27294437909732085,2,1472822600,1.00,1,0.00,1.00,0.00,...,@,1,0.00,@,@,1,@,organic,@,google


In [28]:
train_df_filter.nunique()

channelGrouping                                      8
date                                               366
fullVisitorId                                   723854
visitNumber                                        384
visitStartTime                                  886276
totals_bounces                                       2
totals_hits                                        274
totals_newVisits                                     2
totals_pageviews                                   214
totals_transactionRevenue                         5331
device_browser                                      54
device_deviceCategory                                3
device_isMobile                                      2
device_operatingSystem                              20
geoNetwork_city                                    648
geoNetwork_continent                                 6
geoNetwork_country                                 222
geoNetwork_metro                                    93
geoNetwork

We can see that the time period in question are 366 days (not necessarily consecutive ones -- as days in 'dates' where no visits happened would not be recorded)

In [29]:
train_df_cat.nunique()

fullVisitorId                 714167
device_browser                    54
device_operatingSystem            20
geoNetwork_city                  648
geoNetwork_metro                  93
geoNetwork_country               222
geoNetwork_networkDomain       27520
geoNetwork_region                375
trafficSource_adContent           44
trafficSource_campaign            10
trafficSource_keyword           3507
trafficSource_source             331
trafficSource_referralPath      1260
weekday                            7
dtype: int64

In [30]:
train_df_filter.columns.isin(train_df_cat.columns)

array([False, False,  True, False, False, False, False, False, False,
       False,  True, False, False,  True,  True, False,  True,  True,
        True,  True, False,  True, False, False, False, False,  True,
       False,  True, False,  True,  True])

In [31]:
train_df_filter.iloc[:,train_df_filter.columns.isin(train_df_cat.columns)].nunique()

fullVisitorId                 723854
device_browser                    54
device_operatingSystem            20
geoNetwork_city                  648
geoNetwork_country               222
geoNetwork_metro                  93
geoNetwork_networkDomain       28063
geoNetwork_region                375
trafficSource_adContent           45
trafficSource_campaign            10
trafficSource_keyword           3659
trafficSource_referralPath      1475
trafficSource_source             380
dtype: int64

The "filter and "cat columns do not fully correspond. That is, there is more categorical data in "filter than in "cat, probably because some rows have been dropped in "cat due to too much missing data.

" cat also has helpfully a weekday column.

Probably time of the year might also help -- but then these vary with country (etc.) of which there are 222.

In [ ]:
#train_df_filter.device_operatingSystem.value_counts()

In [ ]:
# pkl = pickle.load('data/data_all_features.pkl')

How many visitors returned?

In [110]:
return_vis = (train_df_filter.fullVisitorId.value_counts() > 1).sum()

In [111]:
return_vis/unique_customers

0.1251053389219373

About 12.5% of all visitors return

In [78]:
customer_rev_visits = rev_by_customer

In [79]:
customer_rev_visits['fullVisitorId_sort'] = customer_rev_visits.index.astype(float) # to sort by customerID one has to convert them to floats as int formats are too low precision for such high numbers

In [80]:
customer_rev_visits.sort_values('fullVisitorId_sort', inplace=True)

In [81]:
customer_rev_visits

,totals_transactionRevenue,fullVisitorId_sort
fullVisitorId,,
4823595352351,0.00,"4,823,595,352,351.00"
5103959234087,0.00,"5,103,959,234,087.00"
10278554503158,0.00,"10,278,554,503,158.00"
20424342248747,0.00,"20,424,342,248,747.00"
26722803385797,0.00,"26,722,803,385,797.00"
...,...,...
11241321903919452424,0.00,"11,241,321,903,919,452,160.00"
35846613770150791963,0.00,"35,846,613,770,150,793,216.00"
42265292326122750572,0.00,"42,265,292,326,122,749,952.00"


In [82]:
visits = pd.DataFrame(train_df_filter.fullVisitorId.value_counts())

In [83]:
visits['fullVisitorId_sort'] = visits.index.astype(float)

In [84]:
visits.sort_values('fullVisitorId_sort', inplace=True)

In [85]:
visits

,fullVisitorId,fullVisitorId_sort
4823595352351,1,"4,823,595,352,351.00"
5103959234087,1,"5,103,959,234,087.00"
10278554503158,1,"10,278,554,503,158.00"
20424342248747,1,"20,424,342,248,747.00"
26722803385797,1,"26,722,803,385,797.00"
...,...,...
11241321903919452424,1,"11,241,321,903,919,452,160.00"
35846613770150791963,1,"35,846,613,770,150,793,216.00"
42265292326122750572,1,"42,265,292,326,122,749,952.00"
47427028546298368646,1,"47,427,028,546,298,372,096.00"


In [86]:
customer_rev_visits['visits'] = visits['fullVisitorId']

In [103]:
paying_customers_visit = customer_rev_visits.sort_values('totals_transactionRevenue', ascending=False).head(paying_customers)

In [105]:
paying_returning = (paying_customers_visit['visits'] > 1).sum() # the majority of paying customers are returning visitors

In [117]:
paying_returning

6610

In [106]:
paying_returning/paying_customers # 65.17%

0.6516809622399684

In [90]:
paying_customers

10143

In [113]:
unique_customers

723854

In [118]:
returning_no_pay = return_vis - paying_returning

In [119]:
no_pay_vis = unique_customers - paying_customers

In [120]:
returning_no_pay/no_pay_vis

0.11762183853128227

Only 11.76% of non-paying visitors return, suggesting that more than one visit (> 1) is a predictor for transactions

In [ ]:
rev_by_customer[rev_by_customer['totals_transactionRevenue'] > 0]

In [ ]:
rev_by_customer_real

How much time passes between visits?

channelGrouping                                      8 
date                                               366
fullVisitorId                                   723854
visitNumber                                        384
visitStartTime                                  886276
totals_bounces                                       2 Total bounces (for convenience). For a bounced session, the value is 1, otherwise it is null.
totals_hits                                        274 Total number of hits within the session. range: 1-446525
totals_newVisits                                     2 Total number of new users in session (for convenience). If this is the first visit, this value is 1, otherwise it is null.
totals_pageviews                                   214 Total number of pageviews within the session.
totals_transactionRevenue                         5331 Total number of pageviews within the session.
device_browser                                      54
device_deviceCategory                                3
device_isMobile                                      2
device_operatingSystem                              20
geoNetwork_city                                    648
geoNetwork_continent                                 6
geoNetwork_country                                 222
geoNetwork_metro                                    93
geoNetwork_networkDomain                         28063
geoNetwork_region                                  375
geoNetwork_subContinent                             23
trafficSource_adContent                             45
trafficSource_adwordsClickInfo.adNetworkType         3
trafficSource_adwordsClickInfo.isVideoAd             1
trafficSource_adwordsClickInfo.page                  9
trafficSource_adwordsClickInfo.slot                  3
trafficSource_campaign                              10
trafficSource_isTrueDirect                           2
trafficSource_keyword                             3659
trafficSource_medium                                 6
trafficSource_referralPath                        1475
trafficSource_source                               380

In [124]:
train_df_filter.totals_hits.value_counts()

1      446525
2      137839
3       70326
4       42379
5       30897
        ...  
333         1
247         1
340         1
259         1
237         1
Name: totals_hits, Length: 274, dtype: int64